Combine data from DMI & Energinet
=======================
This loads in csv file created by the [loadbulkweather.ipynb](loadbulkweather.ipynb) and pulls data from the Energinet API

In [1]:
import pandas as pd

import assetpricing_functions as ap
%load_ext autoreload
%autoreload 2

## Loading DMI data

In [2]:
# path to the CSV file
path = '/Users/johan/Downloads/weather-out/weather_data.csv' 

df_weather = pd.read_csv(path)

In [3]:
# convert to datetime
df_weather['from'] = pd.to_datetime(df_weather['from'])
df_weather['to'] = pd.to_datetime(df_weather['to'])

# adding electricity area to the dataframe
df_weather['area'] = df_weather['cellId'].apply(lambda x: 'DK1' if int(x[-2:]) < 62 else 'DK2') # DK1 is the price area for west Denmark, DK2 is for east Denmark https://energinet.dk/El/Systemydelser/Introduktion-til-Systemydelser/Oversigt-over-systemydelser/

df_weather.head()

In [4]:
# create df with only cellId, geometry type and coordinates
df_geometry = df_weather[['cellId', 'area', 'coordinates']].drop_duplicates()

# plot the polygons
ap.plot_polygons(df_geometry, color_by='area', save='polygon_by_area.png')

Map saved as: output/polygon_by_area.png


## Energinet data

In [5]:
# define url and parameters
base_url = 'https://api.energidataservice.dk/dataset/Elspotprices'
params = {
    'offset': 0,
    'start': '2011-01-01T00:00',
    'end': '2024-10-01T00:00',
    'filter': '{"PriceArea":["DK1"]}' # DK1 is the price area for west Denmark, DK2 is for east Denmark https://energinet.dk/El/Systemydelser/Introduktion-til-Systemydelser/Oversigt-over-systemydelser/
    }

In [14]:
# download the data
df_energy = ap.get_energydata(url=base_url, params=params)

# drop the HourDK column as dmi data is in UTC
df_energy.drop(columns=['HourDK'], inplace=True)

# convert to datetime
df_energy['HourUTC'] = pd.to_datetime(df_energy['HourUTC'])
df_energy.head()


,HourUTC,PriceArea,SpotPriceDKK,SpotPriceEUR
0,2024-09-30 21:00:00,DK1,73.370003,9.840000
1,2024-09-30 20:00:00,DK1,149.199997,20.010000
2,2024-09-30 19:00:00,DK1,163.509995,21.930000
3,2024-09-30 18:00:00,DK1,270.660004,36.299999
4,2024-09-30 17:00:00,DK1,273.119995,36.630001
